In [1]:
import numpy as np
import ciropt as co

In [2]:
problem = co.CircuitOpt()

In [3]:
mu = 0
M = np.inf
f = co.def_function(problem, mu, M)

In [4]:
type(f)

ciropt.function.ConvexFunction

In [5]:
x_star, y_star, f_star = f.stationary_point(return_gradient_and_function_value=True)

In [6]:
h, b, d = problem.h, problem.b, problem.d

In [7]:
R, C = 1, 10


z_1 = problem.set_initial_point()
e2_1 = problem.set_initial_point()
x_1 = co.proximal_step(z_1, f, R/2)[0]
y_1 = (2 / R) * (z_1 - x_1)


e2_2 = e2_1  -  h / (2 * R * C) *  (R * y_1 + 3 * e2_1)  
z_2 = z_1  -  h / (4 * R * C) *  (5 * R * y_1 + 3 * e2_1)
x_2 = co.proximal_step(z_2, f, R/2)[0]
y_2 = (2 / R) * (z_2 - x_2)

v_C1_1 = e2_1 / 2 - z_1
v_C1_2 = e2_2 / 2 - z_2
v_C2_1 = e2_1; v_C2_2 = e2_2 
e1_1 = (e2_1 - R * y_1) / 2

In [8]:
E_1 = (C/2) * (v_C1_1 + x_star)**2 + (C/2) * (v_C2_1)**2
E_2 = (C/2) * (v_C1_2 + x_star)**2 + (C/2) * (v_C2_2)**2
Delta_1 = b * (x_1 - x_star) * (y_1 - y_star) \
        + d * (1/R) * ((e1_1)**2 + (e1_1 - e2_1)**2 + (e2_1)**2)


problem.set_performance_metric(E_2 - (E_1 - Delta_1))

In [9]:
problem.obj = problem.b + problem.d * 1.1

params, sol = problem.solve(solver="ipopt", verbose=False)[:2]

dim_G=5, dim_F=3
Ipopt total # of variables = 42
Actual # of variables = 31


In [10]:
params

{'b': 6.363636345093333, 'h': 6.363636341231115, 'd': 0.2892562160635395}